<a href="https://colab.research.google.com/github/srikarraju/NLP_Project/blob/main/Distant_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ADL/ADL_1/distnt_training_1.6.csv",encoding='ISO-8859-1')
print(df.columns)

In [ ]:
from gensim.models import Word2Vec,KeyedVectors
import gensim.downloader
#model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Colab Notebooks/NLP/GoogleNews_Vectors_300.bin', binary=True)
model = gensim.downloader.load('glove-twitter-50')

[==================================================] 100.0% 199.5/199.5MB downloaded


In [ ]:
def get_train_sentence_vectors(sentences,model):
  words_to_integer_dict = dict()
  integer_to_word_dict = dict()
  print("No. of sentences = ",len(sentences))
  sentence_vectors = []
  max_sentence_length = 0
  word_count = 1
  for sent in sentences:
    vec = []
    sent_length = 0
    for word in sent:
      sent_length += 1
      if words_to_integer_dict.get(word) == None:
        if word in model:
          words_to_integer_dict[word] = word_count
          vec.append(word_count)
          word_count += 1
        else:
          vec.append(0)
      else:
        vec.append(words_to_integer_dict[word])
    max_sentence_length = max(sent_length,max_sentence_length)
    sentence_vectors.append(vec)
  return sentence_vectors,word_count,max_sentence_length,words_to_integer_dict

In [ ]:
import numpy as np
def get_initial_embeddings(model_ft,word_count,words_to_integer_dict):
  count_in_model = 0
  embedding_matrix = [[0]*50]*(word_count+2)  #one for not in model one for not in train data
  for word in words_to_integer_dict:
    if word in model_ft:
      count_in_model += 1
      embedding_matrix[words_to_integer_dict[word]] = model[word]
  embedding_matrix = np.asarray(embedding_matrix,dtype=float)
  print("No. of embeddings in model = ",count_in_model)
  return embedding_matrix

In [ ]:
def pad_input_vectors(sentence_vectors,max_sentence_length):
  for i in range(len(sentence_vectors)):
    sentence_vectors[i] = sentence_vectors[i][0:max_sentence_length]
    for j in range(max_sentence_length-len(sentence_vectors[i])):
      sentence_vectors[i].append(0)
  #print(sentence_vectors[0:2])
  data_x = np.asarray(sentence_vectors)
  return data_x

In [ ]:
# CNN model
from keras.layers import Conv2D, MaxPooling2D, Concatenate, Reshape, Input, Embedding, Dense, Softmax
from keras.models import Sequential,Model
import tensorflow as tf

def get_trained_CNN_model(train_x,train_y,embedding_matrix,word_count,max_sentence_length):
  initializer = tf.keras.initializers.HeNormal()
  input = Input(shape=(max_sentence_length,))
  e = Embedding(word_count+2, 50, weights = [embedding_matrix], input_length=max_sentence_length)(input)
  e.trainable = False
  reshaped_embedding = Reshape((1,max_sentence_length,50,1))(e)
  filter_2 = Conv2D(50,(2,50),activation='relu',kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4))(reshaped_embedding)
  filter_3 = Conv2D(50,(3,50),activation='relu',kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4))(reshaped_embedding) 
  filter_4 = Conv2D(50,(4,50),activation='relu',kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4))(reshaped_embedding)
  reshaped_filter_2 = Reshape((max_sentence_length-1,50,1))(filter_2)
  reshaped_filter_3 = Reshape((max_sentence_length-2,50,1))(filter_3)
  reshaped_filter_4 = Reshape((max_sentence_length-3,50,1))(filter_4)
  max_pooling_2 = MaxPooling2D(pool_size=(max_sentence_length-1,1))(reshaped_filter_2)
  max_pooling_3 = MaxPooling2D(pool_size=(max_sentence_length-2,1))(reshaped_filter_3)
  max_pooling_4 = MaxPooling2D(pool_size=(max_sentence_length-3,1))(reshaped_filter_4)
  concatenated_output = Concatenate()([max_pooling_2,max_pooling_3,max_pooling_4])
  concatenated_output_reshaped = Reshape((1,150))(concatenated_output)
  dense1_output = Dense(50,activation='relu',kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4))(concatenated_output_reshaped)
  dense2_output = Dense(3,activation='relu',kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4))(dense1_output)
  softmax_probs = Softmax()(dense2_output)

  model3 = Model(input,softmax_probs)
  print(model3.summary())
  print("compiling model")
  model3.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  print("fitting model")
  model3.fit(train_x,train_y,epochs=2)
  e.trainable = True
  model3.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  model3.fit(train_x,train_y,epochs=2)
  return model3

In [ ]:
import numpy as np

tweets1 = tweets[0:100000]
tweets1 += tweets[799999:899999]

labels = [0]*100000
labels += [2]*100000
labels1 = np.asarray(labels1)

for i in range(len(tweets1)):
  tweets1[i] = tweets1[i].split(" ")

sentence_vectors,word_count,max_sentence_length,words_to_integer_dict = get_train_sentence_vectors(tweets1,model)

data_x = pad_input_vectors(sentence_vectors,max_sentence_length)

embedding_matrix = get_initial_embeddings(model,word_count,words_to_integer_dict)

CNN_model = get_trained_CNN_model(data_x,labels1,embedding_matrix,word_count,max_sentence_length)

embeddings = CNN_model.layers[1].get_weights()[0]

In [ ]:
import pandas as pd

words = []
word_embeddings = []
for each in words_to_integer_dict:
  words.append(each)
  word_embeddings.append(embeddings[words_to_integer_dict[each]])

output_df = pd.DataFrame({'words':words,'embeds':word_embeddings},index=None)
output_df.to_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Project/distant_training_embeddings.csv',index=False) 


In [ ]:
import pandas as pd
import numpy as np
def get_embeds_matrix(embeds_list):
  embeds_matrix = []
  for i in range(len(embeds_list)):
    embeds_list[i] = str(embeds_list[i][1:len(embeds_list[i])-1])
    array = embeds_list[i].split()
    for j in range(0,len(array)):
      array[j] = float(array[j])
    embeds_matrix.append(array)
  
  return np.asarray(embeds_matrix)

embeds_file = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP/Project/distant_training_embeddings.csv")
words_list = embeds_file['words'].tolist()
embeds_list = embeds_file['embeds'].tolist()
embeds_matrix = get_embeds_matrix(embeds_list)

In [ ]:
import math
def cosine_similarity(vec_1,vec_2):
  norm1 = 0
  for i in range(len(vec_1)):
    norm1 += vec_1[i]**2
  norm2 = 0
  for i in range(len(vec_2)):
    norm2 += vec_2[i]**2
  norm1 = math.sqrt(norm1)
  norm2 = math.sqrt(norm2)
  cosine = 0
  for i in range(len(vec_1)):
    cosine += vec_1[i]*vec_2[i]
  return (cosine/(norm1*norm2))

In [ ]:
print(cosine_similarity(embeds_matrix[words_list.index('good')],embeds_matrix[words_list.index('bad')]))
print(cosine_similarity(model.wv['good'],model.wv['bad']))

In [ ]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2).fit_transform(embeds_matrix)

In [ ]:
print(X_embedded[words_list.index('good')])
print(X_embedded[words_list.index('bad')])